# Setting Pyspark

In [1]:
import findspark
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd

from configs.config import DATA_SOURCE_DIR, PROJECT_DIR
from pyspark.sql import SparkSession
from train.transforms.utils import *
from train.transforms.splitting import stratified_splitting
from train.transforms.outliers import log_transformation
from train.transforms.categorical_data import * 
from train.transforms.categorical_data import *
from train.transforms.correlation import *

In [ ]:
findspark.init()
findspark.find()

spark= SparkSession \
       .builder \
       .master("local") \
       .appName("Feature Engineering") \
       .config("spark.driver.bindAddress", "0.0.0.0") \
       .config("spark.driver.host", "127.0.0.1") \
       .config("spark.driver.port", "4041") \
       .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/31 14:24:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/31 14:24:26 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
24/12/31 14:24:26 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
24/12/31 14:24:26 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
24/12/31 14:24:26 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
24/12/31 14:24:26 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:555)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.netBind(ServerSocketChannelImpl.java:344)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:301)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:141)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandlerConte

In [3]:
df = spark.read.csv(str(DATA_SOURCE_DIR),header=True,escape="\"")

In [4]:
df = cast_incorrect_column_type(df)

# Handle outliers
- Performing log transformation on some feature columns to reduce skewness

In [5]:
df = log_transformation(df, target_cols=["person_age", "person_income", "person_emp_exp", "loan_amnt", "loan_percent_income", "cb_person_cred_hist_length"])

# Transforming categorical data

In [6]:
categorical_cols = find_categorical_cols(df)
pipeline = onehot_encoding_pipeline(df, categorical_cols)
df = pipeline.transform(df)

In [7]:
columns_to_drop = categorical_cols + [col + "_index" for col in categorical_cols]
df = df.drop(*columns_to_drop)

In [8]:
df_pandas = df.toPandas()

In [9]:
for c in categorical_cols:
    encoded_cols = df_pandas[c + "_encoded"].map(lambda x: list(x.toArray()))
    expanded_df = pd.DataFrame(encoded_cols.tolist(), columns=[c + f"_encoded_{i}" for i in range(len(encoded_cols.iloc[0]))])
    df_pandas = pd.concat([df_pandas, expanded_df], axis=1)
    df_pandas = df_pandas.drop(c + "_encoded", axis=1)

In [10]:
df_pandas.head(5)

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status,person_gender_encoded_0,...,person_education_encoded_3,person_home_ownership_encoded_0,person_home_ownership_encoded_1,person_home_ownership_encoded_2,loan_intent_encoded_0,loan_intent_encoded_1,loan_intent_encoded_2,loan_intent_encoded_3,loan_intent_encoded_4,previous_loan_defaults_on_file_encoded_0
0,3.135494,11.183713,0.000000,10.463132,16.02,0.398776,1.386294,561.0,1,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,3.091042,9.415971,0.000000,6.908755,11.14,0.076961,1.098612,504.0,0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,3.258097,9.428592,1.386294,8.612685,12.87,0.364643,1.386294,635.0,1,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3.178054,11.286702,0.000000,10.463132,15.23,0.364643,1.098612,675.0,1,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3.218876,11.099469,0.693147,10.463132,14.27,0.425268,1.609438,586.0,1,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


# Stratified Splitting

In [12]:
train_df, test_df, val_df = stratified_splitting(
    df_pandas, train_size=0.8, test_size=0.1, val_size=0.1, random_state=42
)

Confirm the proportion of class

In [13]:
test_stratified_sampling(
    df_pandas, train_df, test_df, val_df
)

The proportion of class in each set are the same


/Users/haiduong/Documents/mlops-loan-approval-classification/train/transforms/utils.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  source_df_class_proportion = source_df['loan_status'].value_counts()[0] / source_df['loan_status'].value_counts()[1]
/Users/haiduong/Documents/mlops-loan-approval-classification/train/transforms/utils.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  source_df_class_proportion = source_df['loan_status'].value_counts()[0] / source_df['loan_status'].value_counts()[1]
/Users/haiduong/Documents/mlops-loan-approval-classification/train/transforms/utils.py:64: FutureWarning: Series.__g

# Save as HDFS file

In [14]:
save_to_parquet(train_df, str(PROJECT_DIR / "data" / "features" / "feature_engineering_1.h5"), key="train", mode="w")
save_to_parquet(test_df, str(PROJECT_DIR / "data" / "features" / "feature_engineering_1.h5"), key="test")
save_to_parquet(val_df, str(PROJECT_DIR / "data" / "features" / "feature_engineering_1.h5"), key="val")

24/12/26 15:45:29 WARN TransportChannelHandler: Exception in connection from nguyens-mbp-4/192.168.1.27:57342
java.io.IOException: Operation timed out
	at java.base/sun.nio.ch.SocketDispatcher.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:47)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:339)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:293)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:268)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:425)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:254)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelec